# Noisy VQE Simulation of Lithium Hydride (LiH)

This notebook implements the **Variational Quantum Eigensolver (VQE)** algorithm to estimate the **ground state energy of lithium hydride (LiH)** using PennyLane’s quantum chemistry module. The simulation is performed on a **noisy quantum device**, focusing exclusively on **double excitations** within a Unitary Coupled Cluster (UCC) ansatz.

### Key Features:
- **Molecular Hamiltonian Generation**: Constructs the LiH Hamiltonian using the STO-3G basis and maps it to a qubit representation.
- **Reference State**: Prepares the Hartree-Fock (HF) state from the mean-field solution.
- **Ansatz Construction**: Applies a double excitation ansatz derived from the UCCSD method.
- **VQE Optimization**: Minimizes the expectation value of the Hamiltonian using **Gradient Descent**.
- **Results Analysis**:
  - Convergence plot of energy vs. iteration.
  - Visualization of the ground state quantum amplitudes.

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import sys, os, json, time
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from src.vqe_utils import excitation_ansatz, get_optimizer, set_seed, make_run_config_dict, run_signature, find_existing_run, save_run_record, ensure_dirs, build_run_filename
from src.vqe_utils import RESULTS_DIR, IMG_DIR

seed = 0
set_seed(seed)  # Reproducible runs
ensure_dirs()   # Creates results/ and images/, if missing

In [ ]:
# Define the atoms in the LiH molecule
symbols = ["Li", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.0, 0.0, 0.0],  # Lithium atom at the origin
    [0.0, 0.0, 1.6]   # Hydrogen atom positioned 1.6 Å along the z-axis
])

# Define the LiH Hamiltonian and the number of qubits required
# Default STO-3G basis set
basis = "STO-3G"
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0, basis=basis)

# Which orbitals are occupied (1) or unoccupied (0) in the mean-field (Hartree-Fock) solution
electrons = 4  # 1 from H and 3 from Li
hf = qchem.hf_state(electrons=electrons, orbitals=qubits)  # Creates the Hartree-Fock state

# Define the number of required quantum wires / logical qubits
num_wires = qubits

# Create quantum device simulator backend
dev = qml.device("default.mixed",  # Density-matrix simulator (needed for noise)
                 wires=num_wires)


@qml.qnode(dev)  # Transforms exp_energy below into a quantum node
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))

    # Return the expectation value of the molecular Hamiltonian
    return qml.expval(hamiltonian)

# Calculate ⟨ψ_hf| hamiltonian |ψ_hf⟩ in Hartree (Ha) units
# 1 Ha = 27.2 eV
exp_energy(hf)

In [ ]:
singles, doubles = excitations(electrons=electrons, orbitals=qubits)
ansatz_desc = "UCC doubles (DoubleExcitation over doubles)"

depolarizing_prob = 0.02
amplitude_damping_prob = 0.04

# Define the VQE cost function
@qml.qnode(dev, diff_method="finite-diff")
def cost_function(params):
    excitation_ansatz(params, wires=range(num_wires), hf_state=hf, excitations=doubles, excitation_type="double")

    # Simple noise: 2% depolarizing, 4% amplitude damping
    for w in range(num_wires):
        qml.DepolarizingChannel(depolarizing_prob, wires=w)
        qml.AmplitudeDamping(amplitude_damping_prob, wires=w)

    # Measure the expectation value of the Hamiltonian after applying the ansatz:
    # E(theta) = ⟨ψ(theta)| H |ψ(theta)⟩
    return qml.expval(hamiltonian)

# Create a vector of zeros with the same length as the number of double excitations
initial_params = np.zeros(len(doubles), requires_grad=True)

# Confirm the initial energy of the system using the Hartree-Fock state
# This is the starting point for classical optimization
cost_function(initial_params)

In [ ]:
# The optimizer uses automatic differentiation to compute gradients and adjust the parameters
stepsize = 0.2
max_iterations = 50
optimizer_name = "GradientDescent"
opt = get_optimizer(optimizer_name, stepsize=stepsize)

# Build configuration and signature
cfg = make_run_config_dict(
    symbols=symbols,
    coordinates=coordinates,
    basis=basis,
    ansatz_desc=ansatz_desc,
    optimizer_name=optimizer_name,
    stepsize=stepsize,
    max_iterations=max_iterations,
    seed=seed,
    noisy=True,
    depolarizing_prob=depolarizing_prob,
    amplitude_damping_prob=amplitude_damping_prob
)

sig = run_signature(cfg)
fname = build_run_filename("LiH_Noisy", optimizer_name, seed, sig)
existing = find_existing_run(sig)

if existing:
    # Load results if this configuration has been run before
    with open(existing) as f:
        rec = json.load(f)
    print(f"[reuse] Loaded existing run: {existing}")
    energy = rec["energies"]

    # Accept both old "final_params" (list) or "params_by_step" (last entry)
    if "final_params" in rec:
        theta = np.array(rec["final_params"], requires_grad=True)
    elif "params_by_step" in rec and rec["params_by_step"]:
        theta = np.array(rec["params_by_step"][-1], requires_grad=True)
    else:
        # Fallback if absent — compute from scratch
        existing = None
        print("[reuse] No parameters stored; recomputing.")

else:
    print("[compute] No matching run found; optimizing now.")

if not existing:
    # Compute results if this configuration has NOT been ran before
    theta = np.zeros(len(doubles), requires_grad=True)
    energy = [cost_function(theta)]

    # Optimization loop
    for _ in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_function, theta)
        energy.append(prev_energy)

    # Update the final point:
    energy[-1] = float(cost_function(theta))

    run_record = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),  # Date and time of run
        "molecule": symbols,
        "geometry": coordinates.tolist(),
        "basis": basis,
        "electrons": electrons,
        "num_wires": num_wires,
        "ansatz": ansatz_desc,
        "optimizer": {
            "name": optimizer_name,
            "stepsize": stepsize,
            "iterations_planned": max_iterations,
            "iterations_ran": len(energy)-1
        },
        "seed": seed,
        "energies": [float(e) for e in energy],     # List of energies per iteration
        "final_params": [float(x) for x in theta],  # Store only final params
        "config_hash": sig,
        "noisy": True,
        "depolarizing_prob": depolarizing_prob,
        "amplitude_damping_prob": amplitude_damping_prob,
    }

    persisted = save_run_record(fname, run_record)
    print(f"[saved] {optimizer_name}: {fname}")
    print(f"[mirrored] {persisted}")


# Plotting VQE convergence
plt.plot(range(len(energy)), energy)
plt.xlabel('Iteration')
plt.ylabel('Energy (Ha)')
plt.title('LiH VQE Convergence (Noisy, UCC Doubles)')
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/LiH_Noisy_Gradient_Descent.png', dpi=300)
plt.show()

In [ ]:
print(f"Final ground state energy: {energy[-1]:.8f} Ha")

# Optimized angles in the DoubleExcitation gates for first two excitations
print(f"Final parameters: {theta[0]:.8f}, {theta[1]:.8f}")

In [ ]:
@qml.qnode(dev, diff_method="finite-diff")
def ground_state(params):
    excitation_ansatz(
        params,
        wires=range(num_wires),
        hf_state=hf,
        excitations=doubles,
        excitation_type="double",
    )

    # Simple noise: 2% depolarizing, 4% amplitude damping
    for w in range(num_wires):
        qml.DepolarizingChannel(depolarizing_prob, wires=w)
        qml.AmplitudeDamping(amplitude_damping_prob, wires=w)

    # Return the entire quantum statevector
    return qml.state()

final_rho = ground_state(best_params)  # Density matrix
diag_elements = np.diag(final_rho)

threshold = 1e-2  # Recommended smaller threshold to capture all significant amplitudes
non_zero_indices = np.where(np.abs(diag_elements) > threshold)[0]
non_zero_values = diag_elements[non_zero_indices]

# Build the full ket notation string
ket_terms = []
for idx, amp in enumerate(diag_elements):
    binary_state = f"|{idx:0{num_wires}b}⟩"
    if amp > threshold:
        ket_terms.append(f"({amp:.4f}|{idx:0{num_wires}b}⟩)")

ket_notation = " + ".join(ket_terms)
print(f"\nGround state of LiH (diagonal of density matrix):\n|ψ⟩ = {ket_notation}")

In [ ]:
# Prepare labels and amplitudes for the plot
labels = [f"|{i}⟩" for i in non_zero_indices]

# Bar plot
plt.figure(figsize=(12, 6))
plt.bar(labels, non_zero_values.real, color='blue', label='Real')
plt.bar(labels, non_zero_values.imag, color='orange', bottom=non_zero_values.real, label='Imag')
plt.xlabel('Basis States', fontsize=14)
plt.ylabel('Diagonal Element Population', fontsize=14)
plt.title('Ground State of LiH (Noisy VQE)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/LiH_Noisy_Ground_State.png', dpi=300)
plt.show()